In [1]:
import json
import pandas as pd
import numpy as np
import requests
from statistics import mean

header = '/Volumes/GoogleDrive/My Drive/Scouting Original/' #directory of where all the relevant files used in this program are

HEADER = {'X-TBA-Auth-Key': '0x3LtcI75fiT6AaueA5JuaT9QPKhDDtIjd7gGhNwwbi2dFtIz5L5rygjSbpAFKEZ'} #tba authentication key (used to access the api)
BASE_URL = 'https://www.thebluealliance.com/api/v3/' #url for api base (initial page of api with no included specificity)

#importing from the TBI API requires a base url + an additional url segment indicating a specific path to look into

def tba_request(url):
    r = requests.get(BASE_URL+url, headers=HEADER) #uses the requests library to fetch the desired data from the tba api
    data = json.loads(r.text) #read the json file and return the result
    return data

In [2]:
event_key = '2022cafr'
x = tba_request(f'/event/{event_key}/matches') #loads details for each match for the given event from the tba api, needs wifi

In [3]:
match_number = [i['match_number'] for i in x] #creates a list of match numbers for each match in the event
comp_levels = [i['comp_level'] for i in x] #creates a list of comp levels for each match (qm = quals, qf = quarters, sf = semis, f = finals)

red_team_keys = np.array([[(i[3:]) for i in j['alliances']['red']['team_keys']] for j in x]) #team keys (frc[team number]) for red list with 3 elements in each slot
blue_team_keys = np.array([[(i[3:]) for i in j['alliances']['blue']['team_keys']] for j in x]) #same for blue

red_scores = [i['alliances']['red']['score'] if i['alliances']['red']['score'] >= 0 else np.nan for i in x] #score for red in each match or np.nan if the score doesn't exist (that is, the match hasn't been played yet)
#not keeping scores as 0 for incomplete matches bc to the program, that looks like a tie, not an unplayed match
blue_scores = [i['alliances']['blue']['score'] if i['alliances']['blue']['score'] >= 0 else np.nan for i in x] #same for blue

red_rp = [np.nan if i['score_breakdown'] is None else i['score_breakdown']['red']['rp'] for i in x] #number of RP the red alliance got if the match has been played, otherwise np.nan
blue_rp = [np.nan if i['score_breakdown'] is None else i['score_breakdown']['blue']['rp'] for i in x] #same for blue
youtube_link = ['https://www.youtube.com/watch/{}'.format(i['videos'][0]['key']) if (len(i['videos']) > 0) else np.nan for i in x] #a youtube link if the match has been played
#the one line if in all of these is for instances where the match may or may not have been played yet

In [4]:
fixR = np.where(red_team_keys=='2073B', '9973', red_team_keys)
fixB = np.where(blue_team_keys=='2073B', '9973', blue_team_keys)

In [5]:
fixR = fixR.astype(int)
fixB = fixB.astype(int)
red_team_keys=fixR
blue_team_keys = fixB

In [6]:
tba_data = pd.DataFrame(np.array([match_number, comp_levels, red_team_keys[:,0], red_team_keys[:,1], red_team_keys[:,2], blue_team_keys[:,0], blue_team_keys[:,1], blue_team_keys[:,2], red_scores, blue_scores, red_rp, blue_rp, youtube_link],dtype='object').T, columns=['match', 'level', 'r1', 'r2', 'r3', 'b1', 'b2', 'b3', 'red score', 'blue score', 'red rp', 'blue rp', 'video'])
#basically just creates a gigantic dataframe with all of the previously-created lists as columns
tba_data['match'] = tba_data['match'].astype(int) #sets the match to an int type so it can be sorted
tba_data = tba_data.sort_values('match').reset_index().drop('index', axis=1) #sorts mathches in order


In [7]:
tba_data.to_csv(header + f'{event_key} tba data.csv') #save the file as a csv./
